<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Fall2021/blob/main/DataModelAntoineEquation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Water vapor pressure taken from:  http://www.ddbst.com/en/EED/PCP/VAP_C174.php

!wget https://raw.githubusercontent.com/profteachkids/CHE2064_Fall2021/main/WaterVaporPressure.csv

--2021-08-27 11:46:11--  https://raw.githubusercontent.com/profteachkids/CHE2064_Fall2021/main/WaterVaporPressure.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [text/plain]
Saving to: ‘WaterVaporPressure.csv’

WaterVaporPressure. 100%[===================>]   1.27K  --.-KB/s    in 0s      

2021-08-27 11:46:11 (44.3 MB/s) - ‘WaterVaporPressure.csv’ saved [1300/1300]



In [2]:
import jax
import jax.numpy as jnp
import numpy as np
from scipy.optimize import minimize
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

#JAX default is 32 bit - enable 64 bit - double precision
from jax.config import config
config.update("jax_enable_x64", True)

In [3]:
data=np.loadtxt("WaterVaporPressure.csv", delimiter=",")

In [4]:
data

array([[2.9315e+02, 2.4000e+03],
       [3.0315e+02, 4.2660e+03],
       [3.1315e+02, 7.3330e+03],
       [3.2315e+02, 1.2332e+04],
       [3.3315e+02, 1.9932e+04],
       [3.4315e+02, 3.1224e+04],
       [4.1275e+02, 3.5700e+05],
       [4.3285e+02, 6.1300e+05],
       [4.5125e+02, 9.6000e+05],
       [4.6315e+02, 1.2550e+06],
       [4.7335e+02, 1.5620e+06],
       [4.8255e+02, 1.8850e+06],
       [4.9265e+02, 2.2980e+06],
       [5.0415e+02, 2.8500e+06],
       [5.0777e+02, 3.0430e+06],
       [5.2128e+02, 3.8540e+06],
       [5.3518e+02, 4.8510e+06],
       [5.4149e+02, 5.3640e+06],
       [5.5260e+02, 6.3660e+06],
       [5.5797e+02, 6.8990e+06],
       [5.6169e+02, 7.2880e+06],
       [5.7131e+02, 8.3710e+06],
       [6.4595e+02, 2.1674e+07],
       [6.4604e+02, 2.1794e+07],
       [6.4607e+02, 2.1804e+07],
       [6.4709e+02, 2.1336e+07]])

In [5]:
T=data[:,0]
log_Pv = np.log10(data[:,1])

In [6]:
fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=T, y=log_Pv, mode='markers', marker_size=8, marker_color='rgba(0,0,0,0)',
                marker_line_color='rgb(0,0,255)', marker_line_width=1, name='Data')
fig.update_layout(width=800, height=500)
fig.update_layout(xaxis_title='T (K)', yaxis_title='$Log_{10} P\ (Pa)$a')
fig.show()

In [7]:
def model(a, b, c, T):
  return a-b/(T+c)

In [8]:
a_guess = 10.
b_guess = 2000.0
c_guess = 0.0
x0 = [a_guess, b_guess, c_guess]

def sqerror(x, T, log_Pv):
  model_Pv = model(x[0],x[1],x[2], T)
  err = jnp.sum( (model_Pv-log_Pv)**2)
  return err

res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10)
# res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10, method='COBYLA')
# res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10, jac=jax.grad(sqerror))

In [9]:
res

      fun: 0.0005381098266663227
 hess_inv: array([[ 7.09514940e-02, -1.69094896e+00, -4.97362080e+00],
       [-1.69094896e+00,  1.14418006e+02,  1.72178196e+02],
       [-4.97362080e+00,  1.72178196e+02,  4.71206409e+02]])
      jac: array([-8.74099351e-06,  2.76922947e-08, -1.51216227e-07])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 761
      nit: 41
     njev: 150
   status: 2
  success: False
        x: array([  10.20869359, 1756.3577872 ,  -36.02757871])

In [10]:
T_model = jnp.linspace(T[0],T[-1],100)
fig.add_scatter(x=T_model, y=model(*res.x, T_model), mode='lines', line_width=1, name='Antoine')
fig.show()